# Discover items in your storage

[PySTAC](https://pystac.readthedocs.io/en/stable/) is a library for working with [STAC](https://docs.up42.com/developers/api-assets/stac-about). This tutorial explains how to discover items delivered to your storage in the context of a tasking or archive order in 4 simple steps.

## Install dependencies

In [ ]:
!pip install up42-py --upgrade -q

import pathlib
import up42


## Setup your credentials

Create a `credentials.json` file in a directory named `.up42` under your home directory by running the cell below. The path to the file will be `~/.up42/credentials.json`.

In [ ]:
# Define the credentials file path if it doesn't exist
credentials_file_path = pathlib.Path.home().joinpath(".up42/credentials.json")
credentials_file_path.parent.mkdir(parents=True, exist_ok=True)
credentials_file_path.touch(exist_ok=True)

1. Click the link above to the created file and paste the following code:
    ```
    {
    "username": "<your-email-address>",
    "password": "<your-password>"
    }
    ```
2. Retrieve the email address and password used for logging into the console. Use them as values for `username` and `password`.
3. Save the `credentials.json` file.
4. Check that the authentication was successful as follows:

In [ ]:
up42.authenticate(cfg_file=credentials_file_path)

## Authenticate and create a PySTAC client connection

In [ ]:
up42.authenticate(cfg_file=credentials_file_path)
UP42_client = up42.stac_client()

## Find an image from a particular provider in your storage

First retrieve the UP42 product id for the desired product (in this case SPOT) and search for matching items.

In [ ]:
# Retrieve a list of geospatial collections
data_collections = up42.ProductGlossary.get_collections(
    collection_type= up42.CollectionType.ARCHIVE,
)

# Iterate over collections and select SPOT
collection = next(c for c in data_collections if c.name == "spot")

# Find the display data product
data_product = next(
    d for d in collection.data_products if d.name == "spot-display"
    )

# This filter searches for spot-display items in your storage
filter = {
    "op": "=",
    "args": [
        {
            "property": "up42-product:product_id"
        },
        data_product.id
    ]
}

# Perform the search and iterate to next item
stac_items_search = UP42_client.search(filter=filter)
item = next(stac_items_search.items())


## Select the RGB image and download it to your home folder

The selected item contains several assets (provider metadata, quicklooks, cloud mask, RGB image etc). Now it's time to select the RGB multispectral asset.

In [ ]:
# Find asset with desired bands
rgb_bands = {"red", "green", "blue"}
rgb_key= None
for key, asset in item.assets.items():
    bands = asset.extra_fields.get("eo:bands")
    if not bands:
        continue
    band_names = set([band["name"] for band in bands])
    if rgb_bands.issubset(band_names):
        rgb_key = key
        break

rgb_asset = item.assets[rgb_key]

# Download asset to your home folder
rgb_asset.file.download(output_directory=pathlib.Path.home())
